In [2]:
import pandas as pd
import psycopg2
import numpy as np
import json

df = pd.read_csv("Stops.txt")
print(df.columns)
df = df.replace({np.nan: None, "": None}) # Replace empty strings and NaNs with None

def safe_int(val): # Needed convert certain attributes to int value required for database
    try:
        return int(val)
    except (ValueError, TypeError):
        return None

df['stop_code'] = df['stop_code'].apply(safe_int).astype('Int64')
df['wheelchair_boarding'] = df['wheelchair_boarding'].apply(safe_int).astype('Int64')
df['location_type'] = df['location_type'].apply(safe_int).astype('Int64')
df['stop_lat'] = df['stop_lat'].astype(float)
df['stop_lon'] = df['stop_lon'].astype(float)

with open('credentials.json', 'r') as f:
    config = json.load(f)

conn = psycopg2.connect(
    host=config['host'],
    database=config['database'],
    user=config['user'],
    password=config['password']
)
cursor = conn.cursor()

for index, row in df.iterrows():
    stop_id = row['stop_id']
    values = [
        stop_id,
        row['stop_code'],
        row['stop_name'],
        row['stop_lat'],
        row['stop_lon'],
        row['location_type'],
        row['parent_station'],
        row['wheelchair_boarding'],
        row['platform_code']
    ]
    values = [None if pd.isna(v) else v for v in values]
    try:
        cursor.execute("""
            INSERT INTO gtfs_stops (stop_id, stop_code, stop_name, stop_lat, stop_lon, location_type, parent_station, wheelchair_boarding, platform_code)
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
            ON CONFLICT (stop_id) DO NOTHING;
        """, values)
    except Exception as e:
        print(f"Error inserting row {row}: {e}")

conn.commit()
cursor.close()
conn.close()


Index(['stop_id', 'stop_code', 'stop_name', 'stop_lat', 'stop_lon',
       'location_type', 'parent_station', 'wheelchair_boarding',
       'platform_code'],
      dtype='object')


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x83 in position 69: invalid start byte

In [3]:
import requests
import json

def poi_in_bounding_box(xmin, ymin, xmax, ymax, filters={}):
    baseURL = 'https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query'
    params = {
        'geometry': f'{xmin},{ymin},{xmax},{ymax}',
        'geometryType': 'esriGeometryEnvelope',
        'inSR': '4326',
        'spatialRel': 'esriSpatialRelIntersects',
        'outFields': '*',
        'returnGeometry': 'true',
        'f': 'json',
        'where': ' AND '.join([f"{k}='{v}'" for k, v in filters.items()]) if filters else '1=1'
    }
    response = requests.get(baseURL, params=params)
    return json.loads(response.text)['features']



In [ ]:
# Sydney CBD area
xmin, ymin, xmax, ymax = 151.2, -33.88, 151.22, -33.86
pois = poi_in_bounding_box(xmin, ymin, xmax, ymax)
pois


In [ ]:
import geopandas as gpd
import time

sa2 = gpd.read_file('SA2/SA2_2021_AUST_GDA2020.shp')
selected_sa4 = 'Sydney - Blacktown'
sa2_in_sa4 = sa2[sa2['SA4_NAME21'] == selected_sa4]

pois = []
for idx, row in sa2_in_sa4.iterrows():
    xmin, ymin, xmax, ymax = row['geometry'].bounds  
    pois = poi_in_bounding_box(xmin, ymin, xmax, ymax)

    for poi in pois:
        poi['SA2_NAME21'] = row['SA2_NAME21']  
        pois.append(poi)
    time.sleep(1)
pois


In [ ]:
import psycopg2

with open('config.json', 'r') as f:
    config = json.load(f)

conn = psycopg2.connect(
    host=config['host'],
    database=config['database'],
    user=config['user'],
    password=config['password']
)
cursor = conn.cursor()

insert_query = """
INSERT INTO PointsOfInterest (
    objectid, topoid, poigroup, poitype, poiname, poilabel, poilabeltype, poialtlabel,
    poisourcefeatureoid, accesscontrol, startdate, enddate, lastupdate,
    msoid, centroidid, shapeuuid, changetype, processstate, urbanity,
    x, y, sa2_name21
) VALUES (
    %(objectid)s, %(topoid)s, %(poigroup)s, %(poitype)s, %(poiname)s, %(poilabel)s, %(poilabeltype)s, %(poialtlabel)s,
    %(poisourcefeatureoid)s, %(accesscontrol)s, %(startdate)s, %(enddate)s, %(lastupdate)s,
    %(msoid)s, %(centroidid)s, %(shapeuuid)s, %(changetype)s, %(processstate)s, %(urbanity)s,
    %(x)s, %(y)s, %(SA2_NAME21)s
) ON CONFLICT (objectid) DO NOTHING;
""" 

for poi in pois:
    attributes = poi.get('attributes', {})
    geometry = poi.get('geometry', {})
    data = {
        'objectid': attributes.get('objectid'),
        'topoid': attributes.get('topoid'),
        'poigroup': attributes.get('poigroup'),
        'poitype': attributes.get('poitype'),
        'poiname': attributes.get('poiname'),
        'poilabel': attributes.get('poilabel'),
        'poilabeltype': attributes.get('poilabeltype'),
        'poialtlabel': attributes.get('poialtlabel'),
        'poisourcefeatureoid': attributes.get('poisourcefeatureoid'),
        'accesscontrol': attributes.get('accesscontrol'),
        'startdate': attributes.get('startdate'),
        'enddate': attributes.get('enddate'),
        'lastupdate': attributes.get('lastupdate'),
        'msoid': attributes.get('msoid'),
        'centroidid': attributes.get('centroidid'),
        'shapeuuid': attributes.get('shapeuuid'),
        'changetype': attributes.get('changetype'),
        'processstate': attributes.get('processstate'),
        'urbanity': attributes.get('urbanity'),
        'x': geometry.get('x'),
        'y': geometry.get('y'),
        'SA2_NAME21': poi.get('SA2_NAME21')
    }
    cursor.execute(insert_query, data)
conn.commit()
cursor.close()
conn.close()


All POIs inserted into database.


In [ ]:
import geopandas as gpd
from sqlalchemy import create_engine

def load_catchments():
    with open('config.json', 'r') as f:
        config = json.load(f)

    connection_string = f"postgresql+psycopg2://{config['user']}:{config['password']}@{config['host']}:5432/{config['database']}"
    engine = create_engine(connection_string)

    catchment = gpd.read_file('catchments/catchments_secondary.shp')
    catchment.to_postgis(
        name='schoolcatchments',
        con=engine,
        if_exists='append',
        index=False
    )

load_catchments() # Run this for both primary and secondary catchments 

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import psycopg2

sa2 = gpd.read_file('SA2/SA2_2021_AUST_GDA2020.shp')
selected_sa4 = 'Sydney - Eastern Suburbs'
sa2_in_sa4 = sa2[sa2['SA4_NAME21'] == selected_sa4]

def businesses_per_1000():
    with open('config.json', 'r') as f:
        config = json.load(f)

    conn = psycopg2.connect(
        host=config['host'],
        database=config['database'],
        user=config['user'],
        password=config['password']
    )
    # Query returns SA2 names, number of businesses and number of people 
    # Condition checks for if the business is in the set of SA2 within the Eastern Suburbs
    # Selected industry: Health Care and Social Assistance
    query = """
    SELECT B.sa2_name, B.total_businesses, P.total_people
    FROM Businesses B JOIN Population P ON B.sa2_name = P.sa2_name
    WHERE B.sa2_name in (SELECT distinct(sa2_name21) FROM PointsOfInterest) 
    and B.industry_name = 'Health Care and Social Assistance'
    AND P.total_people > 100;
    """
    businesses = pd.read_sql_query(query, conn)
    conn.close()

    businesses_per_1000 = {}
    for idx, row in businesses.iterrows():
        sa2_name = row['sa2_name']
        total_businesses = row['total_businesses']
        total_people = row['total_people']

        per_1000 = (total_businesses / total_people) * 1000
        businesses_per_1000[sa2_name] = per_1000

    return businesses_per_1000 # returns Dictionary with key-value as sa2_name : number of businesses per 1000 people

def stop_counts(sa2_in_sa4):
    with open('config.json', 'r') as f:
        config = json.load(f)

    conn = psycopg2.connect(
        host=config['host'],
        database=config['database'],
        user=config['user'],
        password=config['password']
    )
    query = """
    SELECT *
    FROM gtfs_stops;
    """
    stops = pd.read_sql_query(query, conn)
    conn.close()

    stop_counts = {}
    for idx, row in sa2_in_sa4.iterrows(): 
        sa2_name = row['SA2_NAME21']
        stop_counts[sa2_name] = 0

    for index, stop_row in stops.iterrows(): #Loops through each stop and increments the stop count of the sa2 which it belongs
        stop_lat = stop_row['stop_lat']
        stop_lon = stop_row['stop_lon']
        for idx, sa2_row in sa2_in_sa4.iterrows():
            xmin, ymin, xmax, ymax = sa2_row['geometry'].bounds  
            if (xmin <= stop_lon <= xmax) and (ymin <= stop_lat <= ymax):
                sa2_name = sa2_row['SA2_NAME21']
                stop_counts[sa2_name] += 1
                break  
    return stop_counts # returns Dictionary with key-value as sa2_name : count

def schools_per_1000(sa2_in_sa4):
    with open('config.json', 'r') as f:
        config = json.load(f)

    conn = psycopg2.connect(
        host=config['host'],
        database=config['database'],
        user=config['user'],
        password=config['password']
    )
    # Query returns SA2_name and the sum of young people
    # Only returns SA2 within Eastern suburbs 
    query = """
    SELECT SA2_name, "0_4_people" + "5_9_people" + "10_14_people" + "15_19_people" AS young_people
    FROM Population P
    WHERE sa2_name IN (
    SELECT DISTINCT sa2_name21 FROM PointsOfInterest
    );
    """
    young_population = pd.read_sql_query(query, conn)
    school_query = """
    SELECT * FROM SchoolCatchments;
    """
    school = gpd.read_postgis(school_query, conn, geom_col='geometry') 
    conn.close()

    school_counts = {row['SA2_NAME21']: 0 for idx, row in sa2_in_sa4.iterrows()}

    for idx, school_row in school.iterrows():
        school_geom = school_row['geometry']
        for _, sa2_row in sa2_in_sa4.iterrows():
            sa2_name = sa2_row['SA2_NAME21']
            sa2_geom = sa2_row['geometry']

            if school_geom.intersects(sa2_geom):
                school_counts[sa2_name] += 1
                break

    # Merge school counts with population so we can calculate number of schools per 1000 for each sa2 
    df = pd.DataFrame.from_dict(school_counts, orient='index', columns=['catchment_count'])
    df.index.name = 'sa2_name'
    df = df.reset_index()

    merged = pd.merge(df, young_population, on='sa2_name')
    merged = merged[merged['young_people'] >= 100]
    merged['schools_per_1000'] = merged['catchment_count'] / merged['young_people'] * 1000

    result = dict(zip(merged['sa2_name'], merged['schools_per_1000']))
    return result

def POI_counts():
    with open('config.json', 'r') as f:
        config = json.load(f)

    conn = psycopg2.connect(
        host=config['host'],
        database=config['database'],
        user=config['user'],
        password=config['password']
    )
    # Query returns SA2_name and the number of POI
    # Only return SA2_name in Eastern Suburbs
    # Selected POI_Group: 1 - Civic & Community Services Police Station, Fire Station, Hospital, Library, Post Office, Gaol, Nursing Home
    query = """
    SELECT sa2_name21, Count(*) as Poi_count
    FROM PointsOfInterest
    WHERE sa2_name21 IN (
    SELECT DISTINCT sa2_name21 FROM PointsOfInterest
    )
    AND poigroup = 1
    GROUP BY sa2_name21
    """
    poi_counts = pd.read_sql_query(query, conn)
    conn.close()

    poi_dict = dict(zip(poi_counts['sa2_name21'], poi_counts['poi_count']))
    return poi_dict

def print_dict(d):
    for idx, (key, value) in enumerate(d.items(), start=1):
        print(f"{idx}. {key}: {value}")
    print('')



In [80]:
businesses_per_1000_people = businesses_per_1000()
transport_stop_counts = stop_counts(sa2_in_sa4) # Will take longer than 2mins to run
school_per_1000_people = schools_per_1000(sa2_in_sa4)
POI = POI_counts()

C:\Users\prick\AppData\Local\Temp\ipykernel_7320\3149492226.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  businesses_df = pd.read_sql_query(query, conn)
C:\Users\prick\AppData\Local\Temp\ipykernel_7320\3149492226.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stops_df = pd.read_sql_query(query, conn)
C:\Users\prick\AppData\Local\Temp\ipykernel_7320\3149492226.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  young_population_df = pd.read_sql_query(query, conn)
c:\Users\prick\AppData\Local\Programs\Python\Pyt

In [81]:
def z_score(data_dict):
    series = pd.Series(data_dict)
    z_series = (series - series.mean()) / series.std()
    return z_series.to_dict()

print_dict(z_score(businesses_per_1000_people))
print_dict(z_score(transport_stop_counts))
print_dict(z_score(school_per_1000_people))
print_dict(z_score(POI))

1. Bondi - Tamarama - Bronte: -0.5566006147271904
2. Bondi Beach - North Bondi: -0.7299496729491797
3. Bondi Junction - Waverly: 2.23800214003106
4. Dover Heights: -0.12568752500807368
5. Paddington - Moore Park: 0.059880639815363686
6. Rose Bay - Vaucluse - Watsons Bay: -0.05422308297250848
7. Woollahra: 1.305567509967971
8. Bellevue Hill: 0.3088000933814793
9. Double Bay - Darling Point: 2.236024572442497
10. Kensington (NSW): -0.46651742795181533
11. Kingsford: -0.7181866721975929
12. Maroubra - North: 0.10666027708003535
13. Maroubra - South: -0.9834331159003482
14. Maroubra - West: -0.7833705752282567
15. Randwick - North: -0.4499778146531643
16. Randwick - South: 1.26568844490545
17. Coogee - Clovelly: -0.3023116243340851
18. Malabar - La Perouse: -0.9839590510995185
19. Matraville - Chifley: -1.0022306893338695
20. South Coogee: -0.3641758112682602

1. Bondi - Tamarama - Bronte: 0.9438651368592682
2. Bondi Beach - North Bondi: 0.004143393928267247
3. Bondi Junction - Waverly: 1.

In [ ]:
def Score(x):
    return 1 / (1 + np.exp(-x))

def combined_resource_score(business_z, stop_z, school_z, poi_z):
    final_scores = {}
    all_sa2s = set(business_z) & set(stop_z) & set(school_z) & set(poi_z)

    for sa2 in all_sa2s:
        total_z = business_z[sa2] + stop_z[sa2] + school_z[sa2] + poi_z[sa2]
        final_scores[sa2] = Score(total_z) 
        # final_scores[sa2] = round(Score(total_z), 3) if we want to round to 3 d.p

    return final_scores

businesses_z_score = z_score(businesses_per_1000_people)
stops_z_score = z_score(transport_stop_counts)
schools_z_score = z_score(school_per_1000_people)
POI_z_score = z_score(POI)

scores = combined_resource_score(businesses_z_score, stops_z_score, schools_z_score, POI_z_score)
print_dict(scores)


1. Bellevue Hill: 0.17688169831006928
2. Randwick - North: 0.14024498110595307
3. Kingsford: 0.2610517929515383
4. Rose Bay - Vaucluse - Watsons Bay: 0.6474038986121865
5. Woollahra: 0.40330350266829185
6. South Coogee: 0.014234738649420375
7. Bondi Beach - North Bondi: 0.19142265761390542
8. Double Bay - Darling Point: 0.3436108007793006
9. Kensington (NSW): 0.8002735857779795
10. Maroubra - North: 0.8585431084255251
11. Bondi Junction - Waverly: 0.9975679076430284
12. Maroubra - West: 0.08176661298219916
13. Paddington - Moore Park: 0.9891673129973817
14. Malabar - La Perouse: 0.9757103556501627
15. Maroubra - South: 0.23306570351415054
16. Randwick - South: 0.5639646154123281
17. Coogee - Clovelly: 0.07567447058043619
18. Bondi - Tamarama - Bronte: 0.9722822653505631
19. Dover Heights: 0.5828875248490233

